# Seq2seq
![img](arch.png)

# Teacher forcing
![img](teacher_forcing.png)

In [1]:
!head rus.txt

Go.	Иди.
Go.	Идите.
Hi.	Здравствуйте.
Hi.	Привет!
Hi.	Хай.
Hi.	Здрасте.
Hi.	Здоро́во!
Run!	Беги!
Run!	Бегите!
Run.	Беги!


In [2]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np

/opt/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
num_samples = 50000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'rus.txt'

In [4]:
# Vectorize the data.

START = '\t'
END = '\n'

# fill the following variables:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    
    # lowercase
    # append target text with aux tokens

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 50000
Number of unique input tokens: 50
Number of unique output tokens: 78
Max sequence length for inputs: 19
Max sequence length for outputs: 63


In [6]:
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),    dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),    dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),    dtype='float32')

In [7]:
# TODO
# fill encoder_input_data, decoder_input_data, decoder_target_data

## Define model

In [8]:
from keras import backend
backend.clear_session()

In [9]:
!rm -rf char_seq2seq

In [8]:
latent_dim = 256  # Latent dimensionality of the encoding space.

In [11]:
# TODO build seq2seq model

# Define an input sequence and process it.
encoder_inputs = 
encoder_lstm = 

encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = 
decoder_lstm =
decoder_lstm_outputs = 
decoder_dense = 
decoder_outputs = 


model = 

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 78)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 314368      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  343040      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [13]:
from keras.optimizers import Adam


model.compile( Adam(0.01, decay=0.01), loss='categorical_crossentropy')

In [14]:
from keras import callbacks

batch_size = 256  # Batch size for training.
epochs = 50  # Number of epochs to train for.

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[ callbacks.TensorBoard(log_dir='./char_seq2seq', 
                                            write_graph=True, 
                                           ),
                      callbacks.EarlyStopping(monitor='val_loss', patience=2, ),
                    ],
          validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 28s 701us/step - loss: 0.6763 - val_loss: 0.6643
Epoch 2/50
40000/40000 [==============================] - 27s 669us/step - loss: 0.5150 - val_loss: 0.5839
Epoch 3/50
40000/40000 [==============================] - 27s 678us/step - loss: 0.4594 - val_loss: 0.5383
Epoch 4/50
40000/40000 [==============================] - 27s 672us/step - loss: 0.4286 - val_loss: 0.5116
Epoch 5/50
40000/40000 [==============================] - 27s 664us/step - loss: 0.4086 - val_loss: 0.4945
Epoch 6/50
40000/40000 [==============================] - 27s 669us/step - loss: 0.3946 - val_loss: 0.4847
Epoch 7/50
40000/40000 [==============================] - 27s 677us/step - loss: 0.3837 - val_loss: 0.4736
Epoch 8/50
40000/40000 [==============================] - 27s 663us/step - loss: 0.3749 - val_loss: 0.4664
Epoch 9/50
40000/40000 [==============================] - 28s 710us/step - loss: 0.3677 - val_

/opt/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

In [ ]:
# TODO

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = 

    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    target_seq = 

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        # calculate probabilties for next token
        # TODO
        next_char_prob = 
        sampled_token = 
        decoded_sentence += sampled_token

        # Exit condition: either hit max length
        # or find stop token.
        # TODO

        # Update the target sequence (of length 1).
        # TODO

        # Update states
        # TODO

    return decoded_sentence

![img](BLEU_94.jpg)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu


scores = []

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    
    s = sentence_bleu(target_texts[seq_index], decoded_sentence, weights=(0.25, 0.25, 0.25, 0.25))
    scores.append(s)
    
print('mean {} std {}'.format(np.mean(scores), np.std(scores)))